In [ ]:
# need to install simhash: pip install simhash

import re, os, json
from simhash import Simhash, SimhashIndex

def get_features(s):
    width = 3
    s = s.lower()
    s = re.sub(r'[^\w]+', '', s)
    return [s[i:i + width] for i in range(max(len(s) - width + 1, 1))]

file = os.path.join('..', '..', 'data', 'share-jan-2019.json')

titles = {}
ids = {}

count = 0
with open(file, 'r') as f:
    for line in f:
        data = json.loads(line)
        titles[count] = data['title']
        ids[count] = data['id']        
        
        count += 1
            
objs = [(str(k), Simhash(get_features(v))) for k, v in titles.items()]
index = SimhashIndex(objs, k=3)

similar_docs = {}
for (k, obj) in objs:
    near_dups = index.get_near_dups(obj)
    near_dups.remove(str(k))
    
    if len(near_dups) > 0:
        similar_docs[k] = set(near_dups)

while True:
    no_change = False

    for k, near_dups in similar_docs.items():
        orig_len = len(near_dups)
        for near_dup in near_dups:
            similar_docs[k] = similar_docs[k].union(similar_docs[near_dup])
        new_len = len(similar_docs[k])

        if orig_len != new_len:
            no_change = True

    if no_change == False:
        break  

In [45]:
with open(os.path.join('..', '..', 'data_exploration_results', 'similar_titles.txt'), 'w') as o:
    clustered_docs = set()
    total_clusters = 0
    for k, near_dups in similar_docs.items():
        if k not in clustered_docs:
            o.write('-' * 100 + '\n')

            for near_dup in near_dups:
                o.write('{} -- {}\n'.format(ids[int(near_dup)], titles[int(near_dup)]))
                clustered_docs.add(near_dup)

            clustered_docs.add(k)
            total_clusters += 1

In [46]:
total_clusters

146094